In [ ]:
import os
import json
import urllib.request
import pandas as pd
import numpy as np
import re

# -----------------------------
# Configuration
# -----------------------------
DATA_DIR = "hpo_data"
os.makedirs(DATA_DIR, exist_ok=True)

PHENOTYPE_HPOA_URL = (
    "https://github.com/obophenotype/human-phenotype-ontology/"
    "releases/latest/download/phenotype.hpoa"
)

HP_JSON_URL = (
    "https://github.com/obophenotype/human-phenotype-ontology/"
    "releases/latest/download/hp.json"
)

GENES_TO_DISEASE_URL = (
    "https://github.com/obophenotype/human-phenotype-ontology/"
    "releases/latest/download/genes_to_disease.txt"
)

OMIM_IDS = [
    "OMIM:617892","OMIM:614808","OMIM:615426","OMIM:615515",
    "OMIM:616208","OMIM:616437","OMIM:617921","OMIM:105400",
    "OMIM:205250","OMIM:300857","OMIM:606070","OMIM:606640",
    "OMIM:619133","OMIM:608030","OMIM:608031","OMIM:608627",
    "OMIM:611895","OMIM:612069","OMIM:612577","OMIM:613435",
    "OMIM:613954","OMIM:600795","OMIM:617839","OMIM:619141"
]

def download(url, out_path):
    if not os.path.exists(out_path):
        print(f"Downloading {os.path.basename(out_path)}")
        urllib.request.urlretrieve(url, out_path)
    else:
        print(f"{os.path.basename(out_path)} already exists, skipping download.")

# -----------------------------
# Step 1: Download files
# -----------------------------
phenotype_hpoa_path = os.path.join(DATA_DIR, "phenotype.hpoa")
hp_json_path = os.path.join(DATA_DIR, "hp.json")
genes_to_disease_path = os.path.join(DATA_DIR, "genes_to_disease.txt")

download(PHENOTYPE_HPOA_URL, phenotype_hpoa_path)
download(HP_JSON_URL, hp_json_path)
download(GENES_TO_DISEASE_URL, genes_to_disease_path)


# process the json frequency entries to get numerical values
with open(os.path.join(DATA_DIR, "hp.json")) as f:
    hp = json.load(f)

hp_freq_map = {}

for term in hp["graphs"][0]["nodes"]:
    term_id = term["id"]  # e.g., 'http://purl.obolibrary.org/obo/HP_0040283'
    # Extract just the short HP ID
    hp_id = term_id.split("_")[-1]  # '0040283'
    
    # Look for definition text
    definition = term.get("meta", {}).get("definition", {}).get("val", "")
    
    # Extract percentages using regex
    percents = [float(x) for x in re.findall(r"(\d+\.?\d*)%", definition)]
    
    if len(percents) == 2:
        # Use midpoint of the range
        hp_freq_map[f"HP:{hp_id}"] = np.mean(percents)
    elif len(percents) == 1:
        hp_freq_map[f"HP:{hp_id}"] = percents[0]

# -----------------------------
# Step 2: Load phenotype.hpoa
# -----------------------------
hpoa = pd.read_csv(
    phenotype_hpoa_path,
    sep="\t",
    comment="#",
    header=None,
    names=[
        "OMIM_ID","DiseaseName","Qualifier","HP_ID",
        "Reference","Evidence","Onset","Frequency",
        "Sex","Modifier","Aspect","Biocuration"
    ]
)

# Filter for the ALS OMIM IDs
hpoa = hpoa[hpoa["OMIM_ID"].isin(OMIM_IDS)]
print(f"Filtered phenotype rows: {len(hpoa)}")

# print(df["Frequency"].unique())

# Process Frequency Column
def parse_frequency(freq):
    if pd.isna(freq) or freq == "":
        return pd.Series([np.nan, np.nan, np.nan])

    freq = str(freq).strip()

    # Fraction case
    if "/" in freq:
        try:
            a, t = freq.split("/")
            a, t = float(a), float(t)
            return pd.Series([a, t, (a / t) * 100])
        except ValueError:
            pass

    # HP code case
    if freq.startswith("HP:") and freq in hp_freq_map:
        return pd.Series([np.nan, np.nan, hp_freq_map[freq]])

    # Fallback
    return pd.Series([np.nan, np.nan, np.nan])

# apply to hpoa
hpoa[["affected", "total", "percent"]] = hpoa["Frequency"].apply(parse_frequency)
hpoa["percent"] = hpoa["percent"].round(2)

# -----------------------------
# Step 3: Load hp.json for HP -> name mapping
# -----------------------------
with open(hp_json_path, "r") as f:
    hp_data = json.load(f)

hp_terms = {}
for node in hp_data["graphs"][0]["nodes"]:
    uri = node.get("id", "")
    label = node.get("lbl", None)
    if uri.startswith("http://purl.obolibrary.org/obo/HP_") and label:
        hp_id = uri.split("/")[-1].replace("_", ":")
        hp_terms[hp_id] = label

# Add human-readable phenotype names
hpoa["Phenotype"] = hpoa["HP_ID"].map(hp_terms)

# check hpoa
display(hpoa.head(10))

# -----------------------------
# Step 4: Load genes_to_disease.txt
# -----------------------------
genes_df = pd.read_csv(genes_to_disease_path, sep="\t")

print(genes_df.columns)
display(genes_df)

# Normalize OMIM ID format
# genes_df["OMIM_ID"] = genes_df["disease_id"].astype(str).apply(lambda x: f"OMIM:{x}")
genes_df["OMIM_ID"] = genes_df["disease_id"]

# Filter only the OMIM IDs we care about
genes_df = genes_df[genes_df["OMIM_ID"].isin(OMIM_IDS)]

# Keep just gene symbol + OMIM
# genes_df = genes_df[["gene_symbol","OMIM_ID"]].rename(columns={"geneSymbol": "Gene"})
genes_df = genes_df[["gene_symbol","OMIM_ID"]]

display(genes_df)

# -----------------------------
# Step 5: Merge gene info into phenotype DataFrame
# -----------------------------
df = (
    hpoa
    .merge(genes_df, on="OMIM_ID", how="left")
    [["gene_symbol","OMIM_ID", "DiseaseName", "HP_ID","Phenotype", "affected", "total", "percent"]]
    .sort_values(["gene_symbol","OMIM_ID","HP_ID"])
    .reset_index(drop=True)
)

print(df.head())
print(f"\nTotal rows in final DataFrame: {len(df)}")

# -----------------------------
# Step 6: Save (optional)
# -----------------------------
hpoa.to_csv("als_hpoa.csv", index=False)
genes_df.to_csv("als_genes.csv", index=False)

df.to_csv("als_hpo_gene_phenotypes.csv", index=False)
display(df)

print(f"{df['percent'].isna().sum()} phenotypes have no frequency info")

# Create a boolean mask: True if Phenotype contains 'onset'
onset_mask = df["Phenotype"].str.contains("onset", case=False, na=False)

# Table with only onset-related rows
onset_df = df[onset_mask].copy()

# Table with all other clinical phenotypes
clinical_df = df[~onset_mask].copy()

# save
onset_df.to_csv("als_genes_onset.csv", index=False)
clinical_df.to_csv("als_genes_symptoms.csv", index=False)

print(f"Rows in onset table: {len(onset_df)}")
print(f"Rows in clinical phenotype table: {len(clinical_df)}")
print(f"Diseases in onset table: {onset_df["OMIM_ID"].nunique()}")
print(f"Diseases in clinical table: {clinical_df["OMIM_ID"].nunique()}")

# For each gene, check if all percent values are NaN
genes_all_missing = df.groupby("gene_symbol")["percent"].apply(lambda x: x.isna().all())
# Count how many genes have all NaN
num_genes_all_missing = genes_all_missing.sum()
num_genes = df["gene_symbol"].nunique()
print(f"Number of genes with completely missing percent: {num_genes_all_missing} out of {num_genes}")

# For each gene, check if all percent values are NaN
genes_all_missing = clinical_df.groupby("gene_symbol")["percent"].apply(lambda x: x.isna().all())
# Count how many genes have all NaN
num_genes_all_missing = genes_all_missing.sum()
num_genes = clinical_df["gene_symbol"].nunique()
print(f"Number of genes with completely missing percent, clinical_df: {num_genes_all_missing} out of {num_genes}")


phenotype.hpoa already exists, skipping download.
hp.json already exists, skipping download.
genes_to_disease.txt already exists, skipping download.


C:\Users\William\AppData\Local\Temp\ipykernel_1568\2308695461.py:103: DtypeWarning: Columns (2,6,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  hpoa = pd.read_csv(


Filtered phenotype rows: 294


,OMIM_ID,DiseaseName,Qualifier,HP_ID,Reference,Evidence,Onset,Frequency,Sex,Modifier,Aspect,Biocuration,affected,total,percent,Phenotype
4794,OMIM:615426,Amyotrophic lateral sclerosis 20,NaN,HP:0003155,PMID:23455423,PCS,NaN,NaN,NaN,NaN,P,HPO:probinson[2015-05-10],NaN,NaN,NaN,Elevated circulating alkaline phosphatase conc...
4795,OMIM:615426,Amyotrophic lateral sclerosis 20,NaN,HP:0003560,PMID:23455423,PCS,HP:0003584,NaN,NaN,NaN,P,HPO:probinson[2015-05-10],NaN,NaN,NaN,Muscular dystrophy
4796,OMIM:615426,Amyotrophic lateral sclerosis 20,NaN,HP:0007354,OMIM:615426,IEA,NaN,NaN,NaN,NaN,P,HPO:skoehler[2015-12-30],NaN,NaN,NaN,Amyotrophic lateral sclerosis
4797,OMIM:615426,Amyotrophic lateral sclerosis 20,NaN,HP:0003805,PMID:23455423,PCS,NaN,NaN,NaN,NaN,P,HPO:probinson[2015-05-10],NaN,NaN,NaN,Rimmed vacuoles
4798,OMIM:615426,Amyotrophic lateral sclerosis 20,NaN,HP:0100299,PMID:23455423,PCS,NaN,NaN,NaN,NaN,P,HPO:probinson[2015-05-10],NaN,NaN,NaN,Muscle fiber inclusion bodies
4799,OMIM:615426,Amyotrophic lateral sclerosis 20,NaN,HP:0000006,PMID:23455423,PCS,NaN,NaN,NaN,NaN,I,HPO:probinson[2015-05-10],NaN,NaN,NaN,Autosomal dominant inheritance
12114,OMIM:619141,Frontotemporal dementia and/or amyotrophic lat...,NaN,HP:0000708,PMID:27080313,PCS,NaN,NaN,NaN,NaN,P,HPO:probinson[2021-02-19];HPO:probinson[2021-0...,NaN,NaN,NaN,Atypical behavior
12115,OMIM:619141,Frontotemporal dementia and/or amyotrophic lat...,NaN,HP:0003596,PMID:27080313,PCS,NaN,16/19,NaN,NaN,C,HPO:probinson[2022-08-13];HPO:probinson[2021-0...,16.0,19.0,84.21,Middle age onset
12116,OMIM:619141,Frontotemporal dementia and/or amyotrophic lat...,NaN,HP:0002145,PMID:27080313,PCS,NaN,2/13,NaN,NaN,P,HPO:probinson[2021-02-19],2.0,13.0,15.38,Frontotemporal dementia
12117,OMIM:619141,Frontotemporal dementia and/or amyotrophic lat...,NaN,HP:0001260,PMID:27080313,PCS,NaN,NaN,NaN,NaN,P,HPO:probinson[2021-02-19];HPO:probinson[2021-0...,NaN,NaN,NaN,Dysarthria


Index(['ncbi_gene_id', 'gene_symbol', 'association_type', 'disease_id', 'source'], dtype='object')


,ncbi_gene_id,gene_symbol,association_type,disease_id,source
0,NCBIGene:64170,CARD9,MENDELIAN,OMIM:212050,ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/mim2gene_...
1,NCBIGene:51256,TBC1D7,MENDELIAN,OMIM:248000,ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/mim2gene_...
2,NCBIGene:28981,IFT81,MENDELIAN,OMIM:617895,ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/mim2gene_...
3,NCBIGene:8216,LZTR1,MENDELIAN,OMIM:616564,ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/mim2gene_...
4,NCBIGene:6505,SLC1A1,POLYGENIC,OMIM:615232,ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/mim2gene_...
...,...,...,...,...,...
15778,NCBIGene:55901,THSD1,UNKNOWN,ORPHA:231160,http://www.orphadata.org/data/xml/en_product6.xml
15779,NCBIGene:7049,TGFBR3,UNKNOWN,ORPHA:231160,http://www.orphadata.org/data/xml/en_product6.xml
15780,NCBIGene:1281,COL3A1,UNKNOWN,ORPHA:231160,http://www.orphadata.org/data/xml/en_product6.xml
15781,NCBIGene:83854,ANGPTL6,UNKNOWN,ORPHA:231160,http://www.orphadata.org/data/xml/en_product6.xml


,gene_symbol,OMIM_ID
7,NEK1,OMIM:617892
174,FUS,OMIM:608030
354,ANXA11,OMIM:617839
810,SQSTM1,OMIM:616437
812,PRPH,OMIM:105400
813,DCTN1,OMIM:105400
814,NEFH,OMIM:105400
815,SOD1,OMIM:105400
1062,CHMP2B,OMIM:600795
1304,TUBA4A,OMIM:616208


  gene_symbol   OMIM_ID              DiseaseName               HP_ID               Phenotype             affected  total  percent
0      ANG     OMIM:611895  Amyotrophic lateral sclerosis 9  HP:0000006  Autosomal dominant inheritance    NaN     NaN     NaN   
1      ANG     OMIM:611895  Amyotrophic lateral sclerosis 9  HP:0001257                      Spasticity    NaN     NaN     NaN   
2      ANG     OMIM:611895  Amyotrophic lateral sclerosis 9  HP:0002460          Distal muscle weakness    NaN     NaN     NaN   
3      ANG     OMIM:611895  Amyotrophic lateral sclerosis 9  HP:0003693               Distal amyotrophy    NaN     NaN     NaN   
4      ANG     OMIM:611895  Amyotrophic lateral sclerosis 9  HP:0007354   Amyotrophic lateral sclerosis    NaN     NaN     NaN   

Total rows in final DataFrame: 342


,gene_symbol,OMIM_ID,DiseaseName,HP_ID,Phenotype,affected,total,percent
0,ANG,OMIM:611895,Amyotrophic lateral sclerosis 9,HP:0000006,Autosomal dominant inheritance,NaN,NaN,NaN
1,ANG,OMIM:611895,Amyotrophic lateral sclerosis 9,HP:0001257,Spasticity,NaN,NaN,NaN
2,ANG,OMIM:611895,Amyotrophic lateral sclerosis 9,HP:0002460,Distal muscle weakness,NaN,NaN,NaN
3,ANG,OMIM:611895,Amyotrophic lateral sclerosis 9,HP:0003693,Distal amyotrophy,NaN,NaN,NaN
4,ANG,OMIM:611895,Amyotrophic lateral sclerosis 9,HP:0007354,Amyotrophic lateral sclerosis,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
337,NaN,OMIM:606640,Amyotrophic lateral sclerosis 3,HP:0000726,Dementia,0.0,20.0,0.0
338,NaN,OMIM:606640,Amyotrophic lateral sclerosis 3,HP:0001272,Cerebellar atrophy,0.0,20.0,0.0
339,NaN,OMIM:606640,Amyotrophic lateral sclerosis 3,HP:0002483,Bulbar signs,20.0,20.0,100.0
340,NaN,OMIM:606640,Amyotrophic lateral sclerosis 3,HP:0003581,Adult onset,20.0,20.0,100.0


223 phenotypes have no frequency info
Rows in onset table: 37
Rows in clinical phenotype table: 305
Diseases in onset table: 17
Diseases in clinical table: 23
Number of genes with completely missing percent: 10 out of 24
Number of genes with completely missing percent, clinical_df: 10 out of 24
